In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
K.set_learning_phase(0)

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
from MNISTSeqGen import MNISTSeqGen

In [ ]:
from Model import build_Model

# Create Data Generators

In [ ]:
base_gen = MNISTSeqGen(seq_length=9)
clean_train_gen = base_gen.train_generate(batch_size=32, should_blur=False)
train_gen = base_gen.train_generate(batch_size=32, should_blur=True, probability=0.5)
val_gen = base_gen.val_generate(batch_size=32)

In [ ]:
model_clean = build_Model(num_classes=10, seq_lenght=9)
model_clean.summary()

# Callbacks and Training

In [ ]:
model_checkpoint = ModelCheckpoint(filepath='mnistseq_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)


early = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=5,
                                         verbose=1,
                                         cooldown=0,
                                         min_lr=0.00001)

callbacks = [model_checkpoint,
             early,
             reduce_learning_rate]

model_clean.fit_generator(clean_train_gen, steps_per_epoch=int(base_gen.x_train.shape[0]/32),
                          epochs=25, callbacks=callbacks, validation_data=val_gen,
                          validation_steps=int(base_gen.x_val.shape[0]/32))

# Predictions Visualization

In [ ]:
val_images, val_labels = next(val_gen)
predictions_categorical = model.predict(val_images)

In [ ]:
ind = 6
sample_image, sequence_label_categorical = val_images[ind], val_labels[ind]
prediction = predictions_categorical[ind]
prediction = np.argmax(prediction, axis=1)
sequence_label = np.argmax(sequence_label_categorical, axis=1)
sample_image_plt = sample_image.reshape(28, 252)
plt.imshow(sample_image_plt)
print("image label {}, model prediction {}".format(sequence_label, prediction))

# Train model with blurred data

In [ ]:
blurred_model = build_Model(num_classes=10, seq_lenght=9)

In [ ]:
blurred_model.fit_generator(train_gen, steps_per_epoch=int(base_gen.x_train.shape[0]/32),
                            epochs=25, callbacks=callbacks, validation_data=val_gen,
                            validation_steps=int(base_gen.x_val.shape[0]/32))

# Predictions Visualization
Use the following code after you've trained a model with decent loss/accuracy

In [ ]:
val_blur_gen = base_gen.val_generate(batch_size=32, should_blur=True)
val_images, val_labels = next(val_blur_gen)
predictions_categorical = blurred_model.predict(val_images)

In [ ]:
ind = 0
sample_image, sequence_label_categorical = val_images[ind], val_labels[ind]
prediction = predictions_categorical[ind]
prediction = np.argmax(prediction, axis=1)
sequence_label = np.argmax(sequence_label_categorical, axis=1)
sample_image_plt = sample_image.reshape(28, 252)
plt.imshow(sample_image_plt)
sequence_label, prediction

In [ ]:
blurred_model.evaluate_generator(val_blur_gen, steps=int(base_gen.x_val.shape[0]/32))

# Pretrained model ... on the house
To load a pretrained model

In [ ]:
from keras.models import load_model
pretrained = load_model('mnistseq_epoch-25_loss-0.1905_val_loss-0.1133.h5')


In [ ]:
val_blur_gen = base_gen.val_generate(batch_size=32, should_blur=True)
val_images, val_labels = next(val_blur_gen)
predictions_categorical = pretrained.predict(val_images)

In [ ]:
ind = 3
sample_image, sequence_label_categorical = val_images[ind], val_labels[ind]
prediction = predictions_categorical[ind]
prediction = np.argmax(prediction, axis=1)
sequence_label = np.argmax(sequence_label_categorical, axis=1)
sample_image_plt = sample_image.reshape(28, 252)
plt.imshow(sample_image_plt)
sequence_label, prediction

In [ ]:
pretrained.evaluate_generator(val_blur_gen, steps=int(base_gen.x_val.shape[0]/32))